In [ ]:
import torch
import torch
import torch.utils.data
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import TQDMProgressBar

from livecell_tracker.model_zoo.segmentation.sc_correction import CorrectSegNet
import numpy as np
import warnings
import tqdm
torch.manual_seed(237)


In [ ]:
# torch.cuda.is_available(), torch.cuda.current_device(), torch.cuda.device_count()

import torch
import torch
import torch.utils.data
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import TQDMProgressBar

from livecell_tracker.model_zoo.segmentation.sc_correction import CorrectSegNet
from livecell_tracker.model_zoo.segmentation.sc_correction_dataset import CorrectSegNetDataset
torch.manual_seed(237)

In [ ]:
from pathlib import Path
import pandas as pd
# train_dir = Path("./notebook_results/a549_ccp_vim/train_data_v4/")
# test_dir = Path("./notebook_results/a549_ccp_vim/test_data_v4/")
# train_dir = Path("./notebook_results/a549_ccp_vim/train_data_v5/")
# test_dir = Path("./notebook_results/a549_ccp_vim/test_data_v5/")

# train_dir = Path("./notebook_results/a549_ccp_vim/train_data_v8/")
# test_dir = Path("./notebook_results/a549_ccp_vim/test_data_v8/")
# train_csv = train_dir / "train_data.csv"

# train_csv = "./notebook_results/a549_ccp_vim/train_real_td_data/real_underseg_td1_XY8/data.csv"
# train_csv = "./notebook_results/a549_ccp_vim/train_real_td_data/real_overseg_td1_XY9/data.csv"
# train_csv = "./notebook_results/a549_ccp_vim/train_data_v11/test_overseg_dropout/data.csv"
# train_csv = "/home/ken67/LiveCellTracker-dev/notebooks/notebook_results/a549_ccp_vim/test_data_v11/real_overseg_td1_XY5_dropout/data.csv"
train_csv = "D:\\LiveCellTracker-dev\\datasets\\yaxuan_csn_annotation\\train_data.csv"
# train_csv = "/home/ken67/LiveCellTracker-dev/notebooks/notebook_results/a549_ccp_vim/train_data_v6/synthetic_overseg/data.csv"
# train_csv = "/home/ken67/LiveCellTracker-dev/notebooks/notebook_results/a549_ccp_vim/train_data_v9/synthetic_overseg/data.csv"
train_df = pd.read_csv(train_csv)

if "subdir" not in train_df.columns:
    train_df["subdir"] = "cur"
# test_df = pd.read_csv(test_dir / "train_data.csv")
train_df[:2]

In [ ]:
from livecell_tracker.model_zoo.segmentation.eval_csn import assemble_dataset, assemble_train_test_dataset

# split_seed = 237
# dataset = assemble_dataset(train_df, apply_gt_seg_edt = model.apply_gt_seg_edt, exclude_raw_input_bg=model.exclude_raw_input_bg, input_type=model.input_type)
# train_sample_num = int(len(dataset) * 0.8)
# val_sample_num = len(dataset) - train_sample_num
# split_generator = torch.Generator().manual_seed(split_seed)
# train_dataset, val_dataset = torch.utils.data.random_split(
#     dataset, [train_sample_num, val_sample_num], generator=split_generator
# )

# test_dataset = assemble_dataset(test_df, apply_gt_seg_edt = model.apply_gt_seg_edt, exclude_raw_input_bg=model.exclude_raw_input_bg, input_type=model.input_type)
train_dataset = assemble_dataset(train_df, apply_gt_seg_edt = True, exclude_raw_input_bg=False, input_type="raw_aug_duplicate")

Viz training dataset

In [ ]:
train_dataset.normalize_uint8=False
train_dataset.apply_gt_seg_edt = True

In [ ]:
len(train_dataset), len(train_dataset.raw_img_paths)

In [ ]:
import matplotlib.pyplot as plt
import random

def viz_sample_only(sample):
    fig, axes = plt.subplots(1, 7, figsize=(3 * 7, 3))
    ax_idx = 0
    ax = axes[ax_idx]
    ax.imshow(sample["input"][0])
    ax.set_title("input: dim0")

    ax_idx += 1
    ax = axes[ax_idx]
    ax.imshow(sample["input"][1])
    ax.set_title("input: dim1")

    ax_idx += 1
    ax = axes[ax_idx]
    ax.imshow(sample["input"][2])
    ax.set_title("input:dim2")

    # gt
    ax_idx += 1
    ax = axes[ax_idx]
    ax.imshow(sample["gt_mask"][0])
    ax.set_title("gt0 seg")

    ax_idx += 1
    ax = axes[ax_idx]
    ax.imshow(sample["gt_mask"][1])
    ax.set_title("gt1 seg")

    ax_idx += 1
    ax = axes[ax_idx]
    ax.imshow(sample["gt_mask"][2])
    ax.set_title("gt2 seg")

    ax_idx += 1
    ax = axes[ax_idx]
    ax.imshow(sample["gt_label_mask"])
    ax.set_title("gt label mask")

for i in range(50):
    sample_id = random.randint(0, len(train_dataset) - 1)
    sample = train_dataset[sample_id]
    df_row = train_dataset.raw_df.iloc[sample_id]
    print("source subdir:", df_row["subdir"])
    print("sample id:", sample_id)
    # plt.hist(sample["input"][0].flatten(), bins=100)
    # plt.hist(sample["gt_mask"][1].flatten(), bins=100)
    viz_sample_only(sample)
    plt.suptitle(f"sample id: {sample_id}, raw_df subdir: {df_row['subdir']}, raw_path: {df_row['raw']}")
    # plt.subplots_adjust(top=0.55)
    plt.tight_layout()
    plt.show()
    
    

In [ ]:
# for sample in tqdm.tqdm(train_dataset):
#     gt_label_mask = sample["gt_label_mask"]
#     assert np.unique(gt_label_mask).shape[0] > 0